# Monterey Bay Event Planner Almanac Build

This notebook posts predictions to a Google Calendar based on two existing dataframes in the same directory as this notebook:
* an existing index of obscuration risk ratios per calendar day
* daily 2019 predictions of temperature, humidity, and sky obscuration (generated via my facebook prophet model of the last decade of NOAA Monterey Airport weather data)

In [138]:
# imports
from __future__ import print_function
import datetime
import pickle
import os.path

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar']

### I. Some Preliminary Wrangling

#### Import and One-shift Relative Risk Ratios

In [139]:
risk_ratios = pd.read_csv('calendar_obscuration_risk.csv', index_col=0, names=['risk_ratio'])
risk_ratios.index = risk_ratios.index + 1

In [140]:
risk_ratios.tail()

,risk_ratio
361,0.815217
362,0.983607
363,0.815217
364,0.815217
365,0.815217


#### Import Predictions and Remove Leap Years

In [141]:
predictions = pd.read_csv('predictions.csv', index_col=0, names=['temp', 'hum', 'obsc'], header=0, parse_dates=True)
mask = (predictions.index.date == 29) & (predictions.index.month == 2) # remove leap years
predictions = predictions[~mask]

In [142]:
predictions.tail()

,temp,hum,obsc
2020-03-26,62.913598,61.287053,2.339152
2020-03-27,62.642355,62.222780,2.222179
2020-03-28,62.470312,62.174295,2.342866
2020-03-29,62.418159,61.482002,2.259981
2020-03-30,62.304066,61.380211,2.337992


#### Join Predictions and Ratios into One Frame

In [143]:
# create day of year column
predictions['dayofyear'] = predictions.index.dayofyear
predictions.head()

,temp,hum,obsc,dayofyear
2014-01-01,61.714681,59.445071,2.448811,1
2014-01-02,61.591563,59.762634,2.494193,2
2014-01-03,61.534346,60.957980,2.343700,3
2014-01-04,61.590782,61.165513,2.424949,4
2014-01-05,61.777820,60.719809,2.297179,5


In [144]:
x = predictions
x['dayofyear'] = x['dayofyear'].apply(lambda day: day - 1 if day > 60 else day)

In [145]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2274 entries, 2014-01-01 to 2020-03-30
Data columns (total 4 columns):
temp         2274 non-null float64
hum          2274 non-null float64
obsc         2274 non-null float64
dayofyear    2274 non-null int64
dtypes: float64(3), int64(1)
memory usage: 88.8 KB


In [146]:
# a helper function looks up the risk ratio by the day of the year
def get_risk_ratio_by_dayofyear(row):
    return risk_ratios.loc[row['dayofyear']]

In [147]:
# add risk_ratio to prediction frame
predictions['obscuration_risk_ratio'] = predictions.apply(get_risk_ratio_by_dayofyear, axis=1)

In [148]:
# drop dayof year
predictions = predictions.drop(columns=['dayofyear'])
predictions.head()

,temp,hum,obsc,obscuration_risk_ratio
2014-01-01,61.714681,59.445071,2.448811,1.022727
2014-01-02,61.591563,59.762634,2.494193,1.022727
2014-01-03,61.534346,60.957980,2.343700,0.847458
2014-01-04,61.590782,61.165513,2.424949,1.022727
2014-01-05,61.777820,60.719809,2.297179,1.200000


### II. Generate and Post Each Daily Prediction String for the Rest of 2019

In [149]:
x = predictions

(2274, 4)

#### Setup

In [150]:
# calendar setup
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)

         # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds) # uncomment to sign in

In [151]:
# get a calendar list
page_token = None
while True:
  calendar_list = service.calendarList().list(pageToken=page_token).execute()
  for calendar_list_entry in calendar_list['items']:
    print(calendar_list_entry['accessRole'], ":", calendar_list_entry['summary'])
  page_token = calendar_list.get('nextPageToken')
  if not page_token:
    break

reader : UCSD Grad Programming
reader : Friedensinstallation
writer : CPMC 122
writer : Abjad
owner : Monterey Event Planner Weather Guide
owner : Jeff Trevino's Schedule
owner : nCoda
writer : potocalendar@gmail.com
reader : Holidays in United States
reader : Phases of the Moon
reader : Weather


In [152]:
# get calendar id
el_cid = calendar_list['items'][4]['id']
el_cid

'spud2s1a652omabq2nt605i69o@group.calendar.google.com'

#### Clear Calendar

In [153]:
# service.calendars().clear(calendarId=el_cid).execute() 
# This doesn't work, and it's easier to manually delete the calendar and create a new one.

#### Generate Events and Save Event IDs

#### Define A Function That Does Everything That Must Be Done By Recruiting Some Helpers...

In [154]:
def build_daily_string(row):
    the_string = ''
    the_string += 'temp: ' + '{:.0f}'.format(row['temp']) + ' F ' + '\n'
    the_string += 'hum: ' + '{:.0f}'.format(row['hum']) + '%' + '\n'
    the_string += 'obsc(0-8): ' + '{:.0f}'.format(row['obsc']) + '\n'
    the_string += 'ORR: ' + '{:.2f}'.format(row['obscuration_risk_ratio']) + '\n'
    return the_string

In [155]:
def make_event_body(date_string, the_string):
    body = {'summary': the_string, 
             'location': 'Monterey Airport',
             'description': 'A weather prediction for event planners',
             'start': {
                 'date': date_string
             },
             'end': {
                 'date': date_string
             },
            }
    return body

In [156]:
def make_event_metadata(row, eid, cid, service):
    # define a patch (the info to add in custom fields)
    body = {
      'extendedProperties': {
        'private': {
            'temperature': '{:.0f}'.format(row['temp']) + ' F ',
            'humidity': '{:.0f}'.format(row['hum']) + '%',
            'obscuration(0-8)': '{:.0f}'.format(row['obsc']),
            'obscuration risk ratio': '{:.2f}'.format(row['obscuration_risk_ratio'])
        }
      }
    }
    

In [157]:
def event_from_row(row, cid):
    date_string = str(index.date())
    event_summary_string = build_daily_string(row)
    mr_body = make_event_body(date_string, event_summary_string)
    event = service.events().insert(calendarId=cid, body=mr_body).execute()
#     print('Event created: %s' % (event.get('htmlLink')))
#     so_meta = make_event_metadata(row, event['id'], cid, service)
#     service.events().patch(calendarId=cid, eventId=event['id'], body=so_meta).execute()

#### ...and then Do Everything That Must Be Done

In [158]:
mask = (predictions.index.year >= 2019) & (predictions.index.month >= 9)
predictions = predictions[mask]
predictions.tail()

,temp,hum,obsc,obscuration_risk_ratio
2019-12-27,59.750114,59.649280,3.200220,0.648649
2019-12-28,59.722972,59.820768,3.292152,0.815217
2019-12-29,59.826789,59.372429,3.173228,0.983607
2019-12-30,59.876462,59.532110,3.208347,0.815217
2019-12-31,60.050668,59.008796,2.937333,0.815217


In [164]:
# get ahold of the end of 2019
rest_of_nineteen = predictions.loc['2019-09-01':'2019-12-31']

,temp,hum,obsc,obscuration_risk_ratio
2019-12-27,59.750114,59.649280,3.200220,0.648649
2019-12-28,59.722972,59.820768,3.292152,0.815217
2019-12-29,59.826789,59.372429,3.173228,0.983607
2019-12-30,59.876462,59.532110,3.208347,0.815217
2019-12-31,60.050668,59.008796,2.937333,0.815217


In [165]:
# post predictions for rest of 2019 to calendar
for index, row in rest_of_nineteen.iterrows():
    date_string = str(index.date())
    event_summary_string = build_daily_string(row)
    mr_body = make_event_body(date_string, event_summary_string)
    service.events().insert(calendarId=el_cid, body=mr_body).execute()